In [1]:
import sys
sys.path.insert(1, '../..')
import os
import re
import pandas as pd
from collections import Counter
from medicine import config
# from medicine.script.data_process import DataProcess

In [2]:
file_path = os.path.join(config.Path['data_dir'], '本草纲目(簡體)_Ver2.csv')
data = pd.read_csv(file_path, index_col=[0], engine='python', encoding='utf-8-sig')
data.columns

Index(['目錄', '篇名', '釋名', '內容', '部位', '部位內容', '主治', '氣味', '附方'], dtype='object')

In [3]:
compounds = data.loc[:,'附方']
compounds[11:13]

11    旧一，新一。辟禳时疫∶半天河水，饮之。（《医林集要》）身体白驳∶取树木 \n孔中水洗之，捣桂...
12                                                  NaN
Name: 附方, dtype: object

##### 觀察附方起始文字，並予以清理 (略)
1. 旧一，新一。
2. 新一。
3. 旧一。
4. 新入。
5. （原缺）

In [ ]:
# num_pattern = r'[一二三四五六七八九十]'
# old_pattern = r'旧' + num_pattern + r'{1,3}[\s。]*'
# new_pattern = r'新' + num_pattern + r'{1,3}[\s。]*'
# pattern = old_pattern + '.{1,2}' + new_pattern + \
#     '|' + old_pattern + \
#     '|' + new_pattern + \
#     '|' + '（原缺）' + \
#     '|' + '新入[\s。]*'
# # print(pattern)

# for i in range(len(compounds)):
#     compound = compounds[i]
#     if pd.isnull(compound):
#         continue
#     compound = compound.strip()
# #     # for test
# #     match = re.match(pattern, compound)
# #     if match:
# #         print(match.group())
# #         print(re.sub(pattern, '', compound) + '\n')
# #         pass
# #     else:
# #         print(re.sub(pattern, '', compound) + '\n')
#     compounds[i] = re.sub(pattern, '', compound)


In [21]:
d_len = 2
u_len = 35
# pattern = r'[。）》]{1,3}( ?\w{0,3} ?\n? ?\w{0,8}?[ ，、]{0,2}\w{1,11})∶'
# pattern = r'[。）》]{1,3}((?: ?\w{1,20}[，、]?\n?)?\
# (?:(?:\w+[，、]?){0,2}\n?){0,3}\w{1,11})∶'
# pattern = r'[。）》]{1,3}((?: ?\w{1,20}[ ，、]?\n?)?\
# (?:(?:\w+[，、]){0,2} ?\n?){0,3}\w{1,11})∶'
pattern = r'[。）》]((?: ?\w{0,10}[ ，、]?\n?){0,1}\
(?: ?\w{0,2}[，、] \n){0,2}\
(?:(?:\w+[，、]){1,3} ?\n?){0,3}\w{1,11})∶'


find_list = []
for compound in compounds:
    if pd.isnull(compound):
        continue
    finds = re.findall(pattern, compound)
    if len(finds) > 0:
        finds = [re.sub(r'\s', '', find) for find in finds]
        find_list.extend(finds)
find_list = [find for find in find_list if  d_len <= len(find) <= u_len]
print(len(set(find_list)))
# print(Counter(find_list))
# set(find_list)

5584


In [22]:
print(pattern)

[。）》]((?: ?\w{0,10}[ ，、]?\n?){0,1}(?: ?\w{0,2}[，、] \n){0,2}(?:(?:\w+[，、]){1,3} ?\n?){0,3}\w{1,11})∶


In [15]:
compound_join = '|'.join(find_list)
pattern = '(' + compound_join + ')∶' + '(.*?)' + '(?=(?=(?:' + compound_join + ')∶)|(?<=.$))'

count = 0
for compound in compounds:
    if pd.isnull(compound):
        continue
    finds = re.findall(pattern, re.sub('\s', '', compound))
    if len(finds) > 0:
        print(compound.strip())
        print(finds)
        print()
        
    count += 1
    if count > 2:
        break


新一。灭瘢痕∶以冻凌频熨之，良。（《千金方》）
[('灭瘢痕', '以冻凌频熨之，良。（《千金方》）')]

旧一，新一。辟禳时疫∶半天河水，饮之。（《医林集要》）身体白驳∶取树木 
孔中水洗之，捣桂末唾和敷之，日再上。（张文仲《备急方》）
[('辟禳时疫', '半天河水，饮之。（《医林集要》）'), ('身体白驳', '取树木孔中水洗之，捣桂末唾和敷之，日再上。（张文仲《备急方》）')]



In [16]:
compound_df = pd.DataFrame(columns=['title', 'compound', 'content'])
compound_join = '|'.join(find_list)
pattern = '(' + compound_join + ')∶' + '(.*?)' + '(?=(?=(?:' + compound_join + ')∶)|(?<=.$))'

previous_title = None
for index, row in data.iterrows():
    title = row['篇名']
    compound = row['附方']
    if title == previous_title or pd.isnull(compound):
        continue
    else:
        previous_title = title
    finds = re.findall(pattern, re.sub('\s', '', compound))
    if len(finds) == 0:
        continue
    for find in finds:
        df_row = dict((k, '') for k in compound_df.columns)
        df_row['title'] = title
        df_row['compound'] = find[0]
        df_row['content'] = find[1]
        compound_df = compound_df.append(df_row, ignore_index=True)
    


In [18]:
file_path = os.path.join(config.Path['data_dir'], '本草纲目(簡體)_Ver3.csv')
compound_df.to_csv(file_path, encoding='utf-8-sig')

compound_df.head(3)

,title,compound,content
0,夏冰\n,灭瘢痕,以冻凌频熨之，良。（《千金方》）
1,半天河\n,辟禳时疫,半天河水，饮之。（《医林集要》）
2,半天河\n,身体白驳,取树木孔中水洗之，捣桂末唾和敷之，日再上。（张文仲《备急方》）


#### 去找尋附方內文中符合藥草名稱的pattern
#### <font color='red'>效果不佳</font>

In [19]:
from medicine.models.medicine import Medicine
from medicine.models.alias import Alias 

medicine_all = Medicine.query.all()
alias_all = Alias.query.all()

name_list = list()
for name_len in list(range(9, 1 , -1)):
    m_list = set([m_name.name for m_name in medicine_all if len(m_name.name) == name_len])
    name_list.extend(m_list)
    a_list = set([a_name.name for a_name in alias_all if len(a_name.name) == name_len])
    name_list.extend(a_list)

name_join = '|'.join(name_list)
pattern = '(?=(' + name_join + '))'
for index, row in compound_df.iterrows():
    content = re.sub('\s', '', row['content'])
    match = re.search('（\w{0,3}《?\w{2,8}》?）|《\w{2,8}》', content)
    if match:
        content = re.sub('（\w{0,3}《?\w{2,8}》?）|《\w{2,8}》', '', row['content'])
    find = re.findall(pattern, re.sub('\s', '', content))
    print(row['title'], row['compound'], '\n', row['content'], '\n', set(find), '\n',)
#     print(row['compound'], '\n')



F:\Python\anaconda3\lib\site-packages\sqlalchemy\engine\default.py:470: Warning: (1366, "Incorrect string value: '\\xA5x\\xA5_\\xBC\\xD0...' for column 'VARIABLE_VALUE' at row 480")
  cursor.execute(statement, parameters)


夏冰
 灭瘢痕 
 以冻凌频熨之，良。（《千金方》） 
 set() 

半天河
 辟禳时疫 
 半天河水，饮之。（《医林集要》） 
 {'半天河'} 

半天河
 身体白驳 
 取树木孔中水洗之，捣桂末唾和敷之，日再上。（张文仲《备急方》） 
 set() 

流水
 目不得瞑 
 乃阳气盛不得入于阴，阴气虚，故目不得瞑。治法饮以半夏汤，用流水千里外者八升，扬之万遍，取其清五升煮之，炊苇薪火，置秫米一升，半夏五合，徐炊令竭为一升半，去滓饮汁一小杯，日三饮，以知为度。详半夏下。（《灵枢经》） 
 {'流水', '半夏'} 

流水
 汗后奔豚 
 茯苓桂枝甘草大枣汤，治发汗后脐下悸，欲作奔豚者。茯苓一两，炙甘草二钱半，桂枝三钱，大枣二枚。以甘澜水二升，先煮茯苓，纳诸药煮服之，日三。（张仲景《金匮要略》） 
 {'甘草', '茯苓'} 

流水
 服药过剂烦闷 
 东流水饮一、二升。（《肘后方》） 
 {'流水'} 

井华水
 九窍出血 
 方见主治下。 
 set() 

井华水
 衄血不止 
 叶氏用新汲水，随左右洗足即止，累用有效。 
 set() 

井华水
 一方 
 用冷水面。 
 set() 

井华水
 一方 
 冷水浸纸贴囟上，以熨斗熨之，立止。 
 set() 

井华水
 一方 
 用冷水一瓶，淋射顶上及哑门上。或以湿纸贴之。 
 set() 

井华水
 金疮血出不止 
 冷水浸之即止。（《延寿方》） 
 set() 

井华水
 犬咬血出 
 以水洗，至血止，绵裹之。（《千金方》） 
 set() 

井华水
 蝎虿螫伤 
 以水浸故布拓之，暖即易。（《千金方》）马汗入疮或马毛入疮，肿入腹，杀人。以冷水浸之，频易水，仍饮好酒，立瘥。（《千金方》） 
 set() 

井华水
 鱼骨哽咽 
 取水一杯，合口向水，张口取水气，哽当自下。（《肘后方》） 
 set() 

井华水
 中砒石毒 
 多饮新汲井水，得吐利佳。（《集简方》） 
 set() 

井华水
 中乌喙毒 
 方同上。 
 set() 

井华水
 中蒙汗毒 
 饮冷水即安。（《济急方》） 
 set() 

井华水
 中煤炭毒 
 一时晕倒，不救杀人。急以清水灌之。（唐瑶《经验方》） 
 set() 

井华水
 服药过剂，卒呕不已 
 饮新汲水一升。（《肘后方》） 
 se

 阴股常湿 
 胡粉粉之。（《备急方》） 
 {'胡粉'} 

粉锡
 干湿癣疮 
 方同上。 
 set() 

粉锡
 黄水脓疮 
 官粉（黄）、松香各三钱，黄丹一钱，飞矾二钱。为末，香油二两，熬膏敷之。（邵真人方） 
 {'黄丹', '官粉'} 

粉锡
 小儿耳疮月蚀 
 胡粉，和土涂之。（《子母秘录》） 
 {'胡粉'} 

粉锡
 小儿疳疮 
 熬胡粉，猪脂和涂。（张文仲方） 
 {'胡粉'} 

粉锡
 小儿舌疮 
 胡粉，和猪骨中髓，日三敷之。（《食医心镜》） 
 {'胡粉'} 

粉锡
 燕口吻疮 
 胡粉（炒）一分，黄连半两，为末，敷之。（《普济方》）痘疮瘢痕，或凸或凹。韶粉一两，轻粉一定，和研，猪脂调敷。（陈文中小儿方） 
 {'黄连', '轻粉', '胡粉'} 

粉锡
 妒精阴疮 
 铅粉二钱，银杏仁七个，铜铫内炒至杏黄，去杏取粉，出火毒，研搽效。（《集简方》） 
 {'银杏', '铅粉'} 

粉锡
 反花恶疮 
 胡粉一两，胭脂一两。为末。盐汤洗净敷之，日五次。（《圣惠方》）疮似蜂窠，愈而复发。胡粉、朱砂等分。为末，蜜和涂之。（《圣济录》）血风疮∶《孙氏集效方》∶用官粉四两，水调入碗内，以蕲州艾叶烧烟熏干，入乳香少许同研，香油调作隔纸膏，反复贴之。《杨氏简便方》∶用官粉炒过，桐油调作隔纸贴之。 
 {'官粉', '朱砂', '胡粉'} 

粉锡
 小儿丹毒 
 唾和胡粉，从外至内敷之良。（《千金方》） 
 {'胡粉'} 

粉锡
 汤火烧疮 
 胡粉，羊髓和，涂之。（孙真人方） 
 {'胡粉'} 

粉锡
 疮伤水湿 
 胡粉、炭灰等分，脂和涂孔上，水即出也。（《千金方》）蠼 
 {'胡粉'} 

粉锡
 尿疮 
 酢和胡粉涂之。（《千金方》） 
 {'胡粉'} 

粉锡
 诸蛇螫伤 
 胡粉和大蒜捣涂。（《千金方》） 
 {'大蒜', '胡粉'} 

粉锡
 误吞金银及钱 
 胡粉一两，猪脂调，分再服，令消烊出也。（《外台秘要》） 
 {'胡粉'} 

粉锡
 三年目翳 
 胡粉涂之。（《圣惠方》）口中干燥，烦渴无津。雄猪胆五枚，酒煮皮烂，入定粉一两研匀，丸芡子大，每含化一丸咽汁。（《太平圣惠方》） 
 {'定粉', '胡粉'} 

粉锡
 腹中鳖症 
 胡粉、黍米，淋汁温服，大效。（《卫生易简方》）接骨续筋，止痛活血。定粉、当归各一

 疮不敛 
 方同上。血风疮生脚股上，乃湿毒成风也。黄蜡一两溶化，入银朱一两，搅摊纸上，刺孔贴之。（《简便方》） 
 {'银朱'} 

银朱
 黄水湿疮 
 银朱、盐梅和捣敷之。（《集玄方》） 
 {'银朱'} 

银朱
 癣疮有虫 
 银朱、牛骨髓，桐油调搽。（《医方摘要》） 
 {'银朱'} 

银朱
 头上生虱 
 银朱浸醋，日日梳头。包银朱纸，以碗覆烧之，茶清洗下烟子揉之。包头一夜，至旦虱尽死。（《积德堂方》） 
 {'银朱'} 

灵砂
 伏热吐泻 
 阴阳丸∶用硫黄半两，水银一钱，研黑，姜汁糊丸小豆大。三岁三丸，冷水下；大人三、四十丸。（郑氏《小儿方》） 
 {'水银'} 

灵砂
 诸般吐逆 
 方同上。 
 set() 

灵砂
 霍乱吐逆 
 不问虚实冷热。 
 set() 

灵砂
 二气散，一名青金丹 
 用水银、硫黄等分。研不见星。每服一字至半钱，生姜汤调下。（钱氏《小儿方》） 
 {'生姜', '水银'} 

灵砂
 脾疼反胃 
 灵砂一两，蚌粉一两（同炒赤），丁香、胡椒各四十九粒。为末，自然姜汁煮，半夏粉糊丸梧子大。每姜汤下二十丸。（《普济方》） 
 {'灵砂', '胡椒', '丁香', '半夏'} 

灵砂
 冷气心痛 
 灵砂三分，五灵脂一分。为末，稀糊丸麻子大。每服二十丸，食前石菖蒲、生姜汤下。（《直指方》）九窍出血，因暴惊而得，其脉虚者。灵砂三十粒，人参汤下，三服愈。此证不可错认作血得热则流，妄用凉药误事。（杨仁斋《直指方》）。 
 {'灵砂', '菖蒲', '人参', '五灵脂', '生姜'} 

灵砂
 养正丹 
 又名交泰丹，乃宝林真人谷伯阳方也。却邪辅正，助阳接真。治元气亏虚，阴邪交荡，上盛下虚，气不升降，呼吸不足，头旋气短，心怯惊悸，虚烦狂言，盗汗，腹痛腰痛，反胃吐食，霍乱转筋，咳逆。又治中风涎潮，不省人事，阳气欲脱，四肢厥冷。伤寒阴盛自汗，唇青脉沉。妇人产后月候不匀，带下腹痛。用黑盏一只，入黑铅熔汁，次下水银，次下朱砂末，炒不见星，少顷乃下硫黄末，急搅。有焰，酒醋解之。取出研末，糯粉煮糊丸绿豆大。每服二十丸，盐汤下。四味皆等分。此药升降阴阳，既济心肾，神效。不可具述。（《和济局方》） 
 {'绿豆', '朱砂', '水银'} 

雄黄
 猝中邪魔 
 雄黄末，吹鼻中。（《集验方》） 
 {'雄黄'} 

雄黄
 鬼击成

 冷劳肠泄不止 
 神效太一丹∶禹余粮四两（火醋淬），乌头一两（冷水浸一夜，去皮、脐，焙）。为末，醋糊丸梧子大。每食前温水下五丸。（《圣惠方》）伤寒下痢不止，心下痞硬，利在下焦者，赤石脂禹余粮汤主之。赤石脂、禹余粮各一斤。并碎之，水六升，煮取二升，去滓，分再服。（仲景《伤寒论》） 
 {'乌头', '禹余粮'} 

禹余粮
 赤白带下 
 禹余粮（火醋淬）、干姜等分，赤下干姜减半。为末。空心服二钱匕。（《胜金方》）崩中漏下，青黄赤白，使人无子。禹余粮（研）、赤石脂（研）、牡砺（研）、乌贼骨、伏龙肝（炒）、桂心等分。为末。温酒服方寸匕，日二服。忌葱、蒜。（张文仲《备急方》） 
 {'干姜', '禹余粮', '伏龙肝'} 

禹余粮
 育肠气痛，妇人小腹痛 
 禹余粮为末。每米饮服二钱，日二服，极效。（《卫生易简方》） 
 {'禹余粮'} 

禹余粮
 产后烦躁 
 禹余粮一枚，状如酸馅者，入地埋一半紧筑，炭灰一斤之。湿土罨一宿，打破，去外面石，取里面细者研，水淘五、七度，日干，再研万遍。用甘草汤服二钱，一服立效。（《经验方》） 
 {'甘草', '禹余粮'} 

禹余粮
 身面瘢痕 
 禹余粮、半夏等分。为末，鸡子黄和敷。先以布拭干，勿见风，日三。十日，十年者亦灭。（《圣济录》）大风疠疾，眉发堕落，遍身顽痹。禹余粮二斤，白矾一斤，青盐一斤。为末。罐子固济，炭火一秤之，从辰至戌。候冷研粉，埋土中，三日取出。每一两，入九蒸九曝炒熟胡麻末三两。每服二钱，荆芥茶下，日二服。（《圣惠方》） 
 {'炭火', '禹余粮', '青盐', '胡麻', '半夏'} 

空青
 黑翳覆瞳 
 空青、矾石（烧）各一两，贝子四枚。研细，日点。（《圣济录》） 
 {'空青', '矾石', '贝子'} 

空青
 肤翳昏暗 
 空青二钱，蕤仁（去皮）一两，片脑三钱。细研，日点。（《圣济录》）一切目疾，雀目、赤目、青盲、内外障翳、风眼用此，觉目中凉冷为验。杨梅青（洗净）、胡黄连（洗）各二钱半，槐芽（日未出时勿语采之，入青竹筒内，垂于天、月二德方，候干，勿见鸡犬，为末）一钱半。共末，入龙脑一字密收。每卧时，漱口仰头，吹一字入两鼻内便睡，隔夜便明。（《圣济录》）中风口∶见主治。 
 {'黄连', '胡黄连', '杨梅青', '片脑', '空青'} 

曾青
 斑疮入目不退者 
 曾青一钱，丹砂二钱。为

...

 呃逆不止 
 黄蜡烧烟熏，二三次即止。（《医方摘要》） 
 set() 

蜜蜡
 霍乱吐利 
 蜡一弹丸，热酒一升化服，即止。（《肘后方》） 
 set() 

蜜蜡
 诸般疮毒 
 疮、金疮、汤火等疮。用黄蜡一两，香油二两，黄丹半两，同化开，顿冷，瓶 
 {'黄丹'} 

蜜蜂
 大风疠疾，须眉堕落，皮肉已烂成疮者 
 用蜜蜂子、胡蜂子、黄蜂子（并炒）各一分，白花蛇、乌蛇（并酒浸，去皮、骨，炙干）、全蝎（去土，炒）、白僵蚕（炒）各一两，地龙（去土，炒）半两，蝎虎（全者，炒）、赤足蜈蚣（全者，炒）各十五枚，丹砂一两，雄黄（醋熬）一分，龙脑半钱，上为末。每服一钱匕，温蜜汤调下，日三五服。（《总录》） 
 {'雄黄', '白花蛇', '丹砂', '蜜蜂', '蜈蚣', '乌蛇'} 

土蜂
 面黑令白 
 土蜂子未成头翅者，炒食，并以酒浸敷面。（《圣惠方》） 
 {'土蜂'} 

露蜂房
 小儿卒痫 
 大蜂房一枚，水三升，煮浓汁浴之，日三四次佳。（《千金方》） 
 set() 

露蜂房
 脐风湿肿 
 久不瘥者。蜂房烧末，敷之，效。（《子母秘录》） 
 set() 

露蜂房
 手足风痹 
 黄蜂窠大者一个（小者三、四个）烧灰，独头蒜一碗，百草霜一钱半，同捣敷上。一时取下，埋在阴处。 
 {'百草霜'} 

露蜂房
 忌生冷、风气瘙痒及瘾疹 
 集验方，蜂房（炙）、蝉蜕等分，为末。酒服一钱，日三服。《梅师方》∶用露蜂房煎汁二升，入芒硝敷之，日五次。 
 {'蝉蜕', '露蜂房', '芒硝'} 

露蜂房
 风热牙肿连及头面 
 用露蜂房，烧存性，研末，以酒少许调，噙漱之。（《十便良方》） 
 {'露蜂房'} 

露蜂房
 风虫牙痛 
 露蜂房煎醋，热漱之。《袖珍方》∶用草蜂房一枚，盐实孔内烧过盐汤漱去。或取一块咬之。秘方也。《普济方》∶用露蜂房一个，乳香三块，煎细辛煎水漱之。又露蜂房、全蝎同研，擦之。《圣惠方》∶用蜂房蒂，绵包咬之效。 
 {'细辛', '露蜂房'} 

露蜂房
 喉痹肿痛 
 《普济方》露蜂房灰、白僵蚕等分，为末。每乳香汤服半钱。《食医心镜》∶用蜂房烧灰。每以一钱吹入喉内。不拘大人、小儿。 
 {'露蜂房'} 

露蜂房
 重舌肿痛 
 蜂房炙研，酒和敷之，日三、四次。（《圣惠方》） 
 set() 

露蜂房
 舌上出血窍如针孔 
 用紫金沙

 明目 
 劳伤肝气目暗方∶用萤极妙。一方用白犬胆。（《圣惠方 
 set() 

衣鱼
 小儿胎寒，腹痛汗出 
 用衣中白鱼二七枚，绢包，于儿腹上回转摩之，以愈为度。（《圣惠 
 {'白鱼'} 

衣鱼
 小儿撮口 
 壁鱼儿研末。每以少许涂乳，令儿吮之。（《圣惠》）。 
 set() 

衣鱼
 小儿客忤，项强欲死 
 衣鱼十枚，研敷乳上，吮之入咽，立愈。或以二枚涂母手中，掩儿脐，得吐下愈，外仍以摩儿顶及项强处。《食医心镜》。 
 {'衣鱼'} 

衣鱼
 小儿天吊，目睛上视 
 并口手掣动用壁鱼儿干者十个，湿者五个，用乳汁和研，灌之。（《圣惠方》） 
 {'乳汁'} 

衣鱼
 小儿痫疾 
 白鱼酒∶用衣中白鱼七枚，竹茹一握，酒一升，煎二合，温服之。（《外台》）偏风口∶取白鱼摩耳下，左摩右，右摩左，正乃已。（《孙真人》）。 
 {'白鱼'} 

衣鱼
 小儿重舌 
 衣鱼烧灰，敷舌上。（《千金翼》） 
 {'衣鱼'} 

衣鱼
 目中浮翳 
 书中白鱼末，注少许于翳上，日二。（《外台》） 
 {'白鱼'} 

衣鱼
 沙尘入目不出者 
 杵白鱼，以乳汁和，滴目中，即出。或为末，点之。（《千金》） 
 {'白鱼', '乳汁'} 

衣鱼
 小便不通 
 滑石白鱼散∶用白鱼、滑石、乱发（烧）等分，为散。饮服半钱匕，日三。（《金匮要略》） 
 {'乱发', '白鱼', '滑石'} 

衣鱼
 小便转胞不出 
 纳衣鱼一枚于茎中。（《千金方》） 
 {'衣鱼'} 

衣鱼
 妇人尿血 
 衣中白鱼三十枚，纳入阴中。（《子母秘录》） 
 {'白鱼'} 

鼠妇
 产妇尿秘 
 鼠妇七枚熬，研末，酒服。（《千金翼》） 
 {'鼠妇'} 

鼠妇
 撮口脐风 
 《圣惠》∶用鼠妇虫杵，绞汁少许，灌之。《陈氏》∶生杵鼠妇及雀瓮汁服之。 
 {'雀瓮', '鼠妇'} 

鼠妇
 鹅口白疮 
 地鸡研水涂之，即愈。（《寿域方》） 
 set() 

鼠妇
 风虫牙痛 
 湿生虫一枚，绵裹咬之。勿令人知。（《圣惠》） 
 set() 

鼠妇
 风牙疼痛 
 湿生虫、巴豆仁、胡椒各一枚，研匀，饭丸绿豆大。绵裹一丸咬之，良久涎出吐去，效不可言。（《经效济世方》）痘疮倒∶湿生虫为末，酒服一字，即起。（《痘疹论》） 
 {'巴豆', '胡椒', '绿豆'} 

鼠妇
 蚰蜒入耳 
 湿生

 人咬指烂，久欲脱者 
 鳖甲烧灰敷之。（叶氏《摘玄方》） 
 set() 

蟹
 中鳝鱼毒 
 食蟹即解。（董炳验方） 
 set() 

鲎鱼
 鲎胆散 
 治大风癞疾。用鲎鱼胆、生白矾、生绿矾、腻粉、水银、麝香各半两，研不见星。每服一钱，井华水下。取下五色涎为妙。（《圣济总录》） 
 {'鲎鱼', '水银', '腻粉', '绿矾', '井华水'} 

蚌
 反胃吐食 
 用真正蚌粉，每服称过二钱，捣生姜汁一盏，再入米醋同调送下。（《急救良方》） 
 {'生姜'} 

蚌
 痰饮咳嗽 
 用真蚌粉新瓦炒红，入青黛少许，用淡齑水滴麻油数点，调服二钱。《类编》云∶徽宗时，李防御为入内医官时，有宠妃病痰嗽，终夕不寐，面浮如盘。徽宗呼李治之，诏令供状，三日不效当诛。李忧惶技穷，与妻泣别。 
 {'齑水', '青黛'} 

蚌
 忽闻外叫卖 
 咳嗽药一文一帖，吃了即得睡。李市十帖视之，其色浅碧。恐药性犷悍，并三服自试之，无他。乃取三帖为一，入内授妃服之。是夕嗽止，比晓面消。内侍走报，天颜大喜，赐金帛值万缗。李恐索方，乃寻访前卖药人，饮以酒，浓价求之，则此方也。云自少时从军，见主帅有此方，剽得以度余生耳。 
 {'嗽药'} 

蚌
 痈疽赤肿 
 用米醋和蚌蛤灰涂之，待其干，即易之。（《千金》） 
 set() 

蚌
 雀目夜盲 
 遇夜不能视物。用建昌军螺儿蚌粉三钱，为末，水飞过，雄猪肝一叶，披开纳粉扎定，以第二米泔煮七分熟，仍别以蚌粉蘸食，以汁送下。一日一作。与夜明砂同功。（《直指方》） 
 set() 

蚌
 脚指湿烂 
 用蚌蛤粉干搽之。（《寿域》） 
 set() 

蚌
 积聚痰涎 
 结于胸膈之间，心腹疼痛，日夜不止，或干呕哕食者，炒粉丸主之。用蚌粉一两，以巴豆七粒同炒赤，去豆不用，醋和粉丸梧子大。每服二十丸，姜酒下。丈夫脐腹痛，茴香汤下。女人血气痛，童便和酒下。（孙氏《仁存方》） 
 {'巴豆', '人血'} 

蚬
 卒嗽不止 
 用白蚬壳捣为细末。以熟米饮调，每服一钱，日三服，甚效。（出《圣惠方》） 
 set() 

蚬
 痰喘咳嗽 
 用白蚬壳（多年陈者）烧过存性，为极细末。以米饮调服一钱，日三服。（《急救方》） 
 set() 

蚬
 反胃吐食 
 用黄蚬壳并田螺壳（并取久在泥中者）各炒成白灰。每田螺壳灰二两，黄蚬壳灰一两，入白梅肉四个，

 马出黑汗 
 水化干酪灌之。（藏器） 
 set() 

酥
 蜂螫 
 用酥涂之，妙。（《圣惠》） 
 set() 

酥
 虫咬 
 以酥和盐涂之。（《圣惠方》） 
 set() 

酥
 眯目 
 以酥少许，随左右纳鼻中。垂头卧少顷，令流入目中，物与泪同出也。（《圣济总录》） 
 set() 

醍醐
 风虚湿痹 
 醍醐二两，温酒一杯，每服和醍醐一匙，效。（《心镜》） 
 {'醍醐'} 

醍醐
 中风烦热皮肤瘙痒 
 醍醐四两，每服半匙，温酒一中盏和服，日一。（《圣惠方》） 
 {'醍醐'} 

醍醐
 一切肺病咳嗽脓血不止 
 用好酥五十斤，炼三遍，停凝当出醍醐。每服一合，日三服，以瘥为度，神效。（《外台方》） 
 {'醍醐'} 

醍醐
 小儿鼻塞不通，不能食乳 
 刘氏∶用醍醐二合，木香、零陵香各四分，汤煎成膏。涂头上，并塞鼻中。（《外台》） 
 {'醍醐'} 

乳腐
 血痢不止 
 乳腐一两，浆水一钟，煎服。（《普济方》） 
 {'乳腐', '浆水'} 

阿胶
 瘫缓偏风 
 治瘫缓风及诸风，手脚不遂，腰脚无力者。驴皮胶微炙熟。先煮葱豉粥一升，别贮。又以水一升，煮香豉二合，去滓入胶，更煮七沸，胶烊如饧，顿服之。及暖，吃葱豉粥。如此三四剂即止。若冷吃粥，令人呕逆。（《广济方》） 
 set() 

阿胶
 肺风喘促 
 涎潮眼窜。用透明阿胶切炒，以紫苏、乌梅肉（焙研）等分，水煎服之。（《直指》） 
 {'阿胶'} 

阿胶
 老人虚秘 
 阿胶（炒）二钱，葱白三根。水煎化，入蜜二匙，温服。胞转淋∶阿胶三两，水二升，煮七合，温服。（《千金方》） 
 {'阿胶'} 

阿胶
 赤白痢疾 
 黄连阿胶丸∶治肠胃气虚，冷热不调，下痢赤白，里急后重，腹痛口渴，小便不利。用阿胶（炒过，水化成膏）一两，黄连三两，茯苓二两。为末，捣丸梧子大。每服五十丸，粟米汤下，日三。（《和剂局方》） 
 {'黄连', '阿胶', '茯苓', '粟米', '小便'} 

阿胶
 吐血不止 
 《千金翼》∶用阿胶（炒）二两，蒲黄六合，生地黄三升，水五升，煮三升，分三服。《经验》∶治大人、小儿吐血。用阿胶（肺损呕血并开胃∶用阿胶（炒）三钱，木香一钱，糯米一合半，为末。每服一钱，百沸汤点服一盏，入生地黄汁一合，煎至六分，温服。急以帛系两乳。（《圣惠》） 
 {'地黄', '阿胶',

 {'人屎'} 

人屎
 疔肿初起 
 刮破，以热屎尖敷之，干即易。不过十五遍，即根出立瘥。（《千金》） 
 set() 

人屎
 五色丹毒 
 黄龙汤饮二合，并涂之，良。（《千金方》） 
 set() 

人屎
 九漏有虫 
 干人屎、干牛屎，隔绵贴之，虫闻其气即出。若痒则易之，虫尽乃止。（《千金》） 
 {'人屎'} 

人屎
 疳蚀口鼻，唇颊穿者 
 绵裹人屎贴之，必有虫出。（《十便良方》） 
 {'人屎'} 

人屎
 小儿唇紧 
 人屎灰敷之。（崔知悌方） 
 {'人屎'} 

人屎
 产后阴脱 
 人屎炒赤为末，酒服方寸匕，日三服。（《千金方》） 
 {'人屎'} 

人屎
 鬼舐头疮 
 取小儿粪，和腊猪脂敷之。（《千金方》） 
 set() 

人屎
 金疮肠出 
 干人屎末粉之，即入。（《千金方》） 
 {'人屎'} 

人屎
 针疮血出不止 
 用人屎烧研，敷之。（《千金方》） 
 {'人屎'} 

人屎
 马血入疮肿痛 
 用人粪一鸡子大服之，并涂之。（《千金方》） 
 {'人粪'} 

人屎
 毒蛇咬螫 
 人屎浓封之，帛裹即消。（《千金》）蛊毒百毒及诸热毒，时气热病，口鼻出血。用人屎尖七枚烧灰，水调顿服，温覆取汗即愈。勿轻此方，神验者也。（《外台秘要》） 
 {'人屎'} 

人屎
 诸毒猝恶，热闷欲死者 
 新粪汁，水和服。或干者烧末，渍汁饮。名破棺汤。（苏恭）解药箭毒，毒箭有三种∶交广夷人用焦铜作箭镞，岭北诸处以蛇毒螫物汁着筒中渍箭镞，此二种才伤皮肉，便洪脓沸烂而死。若中之，便饮汁并涂之，惟此最妙。又一种用射罔煎涂箭镞，亦宜此方。（姚僧坦《集验方》） 
 set() 

人屎
 野葛芋毒、山中毒菌欲死者 
 并饮粪汁一升，即活。（《肘后方》） 
 set() 

人屎
 漏肉脯毒 
 人屎烧灰，酒服方寸匕。（《肘后方》） 
 {'人屎'} 

人屎
 恶犬咬伤 
 左盘龙（即人屎也）浓封之，数日即愈。（《蔺氏经验方》） 
 set() 

人屎
 心腹急痛欲死 
 用人屎同蜜擂匀，新汲水化下。（《生生编》） 
 {'人屎'} 

人尿
 头痛至极 
 童便一盏，豉心半合，同煎至五分，温服。（《圣济总录》） 
 set() 

人尿
 热病咽痛 
 童便三合，含之即止。（《圣惠方》） 
 set() 

人尿
 骨蒸发热 
 三岁童便

In [ ]:
medicine_all = Medicine.query.all()
# m_set = set([medicine for medicine in medicine_all if len(medicine.name) == 1])
# sorted(m_set, key=lambda Medicine: Medicine.radical)

# [medicine for medicine in medicine_all if medicine.radical in ['草'] and len(medicine.name) == 1]
sorted([medicine for medicine in medicine_all if len(medicine.name) == 1], key=lambda Medicine: Medicine.radical)